## Predicting Airlines Stocks based on Oil Prices

#### 1. Include all libraries

In [ ]:
library(sqldf)

#### 2. Read in all datasets

In [4]:
setwd('/Users/birupakhya/Documents/Projects/SDMproject/')
delta = read.csv('Deltastocks.csv', skip = 0, header = T, as.is = T, na.strings = "NA")
jetfuelprices = read.csv('jetfuelprices.csv', skip = 12, header = T, as.is = T, na.strings = "#N/A")
brentprices = read.csv('DCOILBRENTEU.csv',skip = 0, header = T, as.is = T, na.strings = ".")

### Data Quality Check

#### 3. Check Missing values

In [2]:
#check missing values
colSums(is.na(delta))
colSums(is.na(jetfuelprices))
colSums(is.na(brentprices))

Date      Open      High       Low     Close    Volume Adj.Close 
        0         0         0         0         0         0         0

DATE VALUE 
    0    91

DATE DCOILBRENTEU 
           0           35